# Zajęcia z programowania 33

## Lista umiejętności

- przeszukiwanie listy w poszukiwaniu konkretnego elementu
- znajdywanie elementu minimalnego
- znajdywanie elementu maksymalnego
- znajdywanie powtórzeń elementów
- znajdywanie par o danej charakterystyce
- znajdywanie NWD, NWW

### Do nauczenia

- transformacja danych
- podróż po grafach
- symulacje warunkowe

## Zadanie

Żółw porusza się po trasie wyznaczonej przez boki sześciokąta. Pierwszy bok przemierza z prędkością
1, drugi i każdy parzysty z prędkością o 3 większą od poprzedniej wartości, trzeci i każdy kolejny
nieparzysty z prędkością o 1 mniejszą od poprzedniej. Po każdych dwóch okrążeniach żółw jest
zmęczony i zamiast o 1 zmniejsza swą prędkość o 10.

Napisz jednoparametrową funkcję `ile(n)`, której wynikiem będzie prędkość żółwia na n-tym boku.
Parametr n może przyjmować wartości od 1 do 10 000.

### Przykłady
- Wynikiem `ile(4)` jest `6`.
- Wynikiem `ile(14)` jest `7`.
- Wynikiem `ile(100)` jest `30`.

## Zadanie z robotem

Robot porusza się od lewego do prawego końca korytarza. Został tak zaprogramowany, że wykonuje tylko dwie podstawowe czynności: idzie naprzód o 1 krok lub zawraca, gdy nie może pójść naprzód. Na początku robot stoi w lewym końcu korytarza, jest skierowany w kierunku prawego i jest naładowany.
Po uruchomieniu porusza się do drugiego końca, a potem zawraca. Wykonując czynności (ruch lub zawracanie) traci energię, więc co jakiś czas musi zostać doładowany. Doładowanie nie jest jednak pełne, każde kolejne umożliwia wykonanie o jedną czynność mniej niż poprzednie.
Napisz dwuparametrową funkcję robot. Pierwszy parametr określa liczbę czynności, jaką robot może wykonać na pełnym naładowaniu, drugi długość korytarza wyrażoną w krokach robota. Oba parametry są liczbami naturalnymi od 1 do 1000. Wynikiem funkcji jest odległość robota od lewego końca po wykonaniu wszystkich możliwych czynności.

### Przykład 1

![](./images/robot1.png)

Dla wartości pierwszego naładowania równej 5 i długości korytarza 10.

Robot po pierwszym naładowaniu idzie 5 kroków, po drugim naładowaniu 4 kroki, po trzecim 1 krok, zawraca i idzie 1 krok, po czwartym naładowaniu 2 kroki, a po ostatnim 1 krok. Odległość od lewego końca wynosi 6.

### Przykład 2

![](./images/robot2.png)

Dla wartości pierwszego naładowania równej 6 i długości korytarza 7.

Robot po pierwszym naładowaniu idzie 6 kroków, po drugim naładowaniu 1 krok, zawraca, a potem 3 kroki, po trzecim 4 kroki, po czwartym zawraca i idzie 2 kroki, po piątym idzie 2 kroki, a po ostatnim 1. Odległość od lewego końca wynosi 5.

# Rozwiązanie analityczne

Poniżej przedstawię szereg rozwiązań problemu, z zamiarem dotarcia do formy analitycznej (pozbawionej pętli). Zaproponuję szereg zapewnień:

```python
# dwa początkowe zapewnienia
assert robot(5,10) == 6 
assert robot(6,7) == 5
# jeśli robot ma baterię 1, to wykona 1 krok
assert robot(1,10) == 1
assert robot(1,100) == 1
assert robot(1,1000) == 1
# jeśli robot ma baterię 2, to wykona 3 kroki
assert robot(2,10) == 3
assert robot(2,100) == 3
assert robot(2,1000) == 3
#jeśli do tego długość korytarza jest mała, to się cofnie.
assert robot(2,3) == 3
assert robot(2,2) == 2
assert robot(2,1) == 0
# dla baterii 3 robot wykona 6 kroków
assert robot(3,6) == 6
assert robot(3,5) == 5
assert robot(3,4) == 3
# dla baterii 4 robot wykona 10 kroków. Jeśli szerokość jest 2, robot wykona 6 kroków na jeden cykl, po czym wyląduje na 1
assert robot(4,2) == 1
```

### Rozwiązanie naiwne

In [26]:
def robot(bateria, dlugosc):
    # Pozycja początkowa
    pozycja = 0
    # Kierunek (1=prawo, 0=lewo)
    kierunek = 1
    # Powtarzamy tak długo dopóki bateria nie jest zużyta
    while bateria > 0:
        naladowanie = bateria
        # Powtarzamy tak długo dopóki nie rozładujemy bieżącej baterii
        while naladowanie > 0:
            # Idziemy w prawo
            if kierunek == 1:
                # Jeśli docieramy do skraju korytarza, zawracamy
                if pozycja == dlugosc:
                    kierunek = 0
                # Jeśli nie, idziemy do przodu
                else:
                    pozycja += 1
            # Idziemy w lewo
            else:
                # Jeśli dotarliśmy do skraju korytarza, zawracamy
                if pozycja == 0:
                    kierunek = 1
                # Jeśli nie, idziemy do tyłu
                else:
                    pozycja -= 1
            # Bateria rozładowuje nam się o krok
            naladowanie -= 1
        # Bateria traci pojemność o 1 punkt
        bateria -= 1
    # Zwracamy pozycję.
    return pozycja

assert robot(5,10) == 6 
assert robot(6,7) == 5
assert robot(1,10) == 1
assert robot(1,100) == 1
assert robot(1,1000) == 1
assert robot(2,10) == 3
assert robot(2,100) == 3
assert robot(2,1000) == 3
assert robot(2,3) == 3
assert robot(2,2) == 2
assert robot(2,1) == 0
assert robot(3,6) == 6
assert robot(3,5) == 5
assert robot(3,4) == 3
assert robot(4,2) == 1

Powyższe rozwiązanie jest dobre, ale zrobione bez pomysłu - implementujemy proste ruchy robota, przedstawione bezpośrednio w zadaniu.

Poniżej zaproponuję pomysł korzystający z faktu, że akt ładowania i chwilowego rozładowania robota nie wpływa na nasz ruch - jeśli wiemy, że nasza bateria ma maksymalną pojemność 5, to znaczy że wykonamy 5+4+3+2+1 = 15 ruchów. W ten sposób możemy zredukować naszą podwójną pętlę do pojedynczej.

In [21]:
def robot(bateria, dlugosc):
    # Pozycja początkowa
    pozycja = 0
    # Kierunek (1=prawo, 0=lewo)
    kierunek = 1
    
    # Obliczamy maksymalne naladowanie - skrotowo byłaby to sum([i for i in range(1, bateria+!)])
    naladowanie = 0
    for i in range(1,bateria+1):
        naladowanie += i
    # Powtarzamy tak długo dopóki nie zużyjemy wszystkich kroków.
    while naladowanie > 0:
        # Idziemy w prawo
        if kierunek == 1:
            # Jeśli docieramy do skraju korytarza, zawracamy
            if pozycja == dlugosc:
                kierunek = 0
            # Jeśli nie, idziemy do przodu
            else:
                pozycja += 1
        # Idziemy w lewo
        else:
            # Jeśli dotarliśmy do skraju korytarza, zawracamy
            if pozycja == 0:
                kierunek = 1
            # Jeśli nie, idziemy do tyłu
            else:
                pozycja -= 1
        # Bateria rozładowuje nam się o krok
        naladowanie -= 1
    # Zwracamy pozycję.
    return pozycja

assert robot(5,10) == 6 
assert robot(6,7) == 5
assert robot(1,10) == 1
assert robot(1,100) == 1
assert robot(1,1000) == 1
assert robot(2,10) == 3
assert robot(2,100) == 3
assert robot(2,1000) == 3
assert robot(2,3) == 3
assert robot(2,2) == 2
assert robot(2,1) == 0
assert robot(3,6) == 6
assert robot(3,5) == 5
assert robot(3,4) == 3
assert robot(4,2) == 1

Udało nam się zredukować program o jedną pętlę, jednak dalej jest on nieefektywny - gdybyśmy mieli olbrzymią baterię (na 1000 kroków), oznaczałoby to przymus wykonania 1,000,000 powtórzeń. Warunki zadania są tak skonstruowane że nie musimy się przejmować złożonością obliczeniową (milion obrotów wykonuje się w sekundę), jednak można przemyśleć czy nie ma prostszego rozwiązania.

Poniżej zaproponuję następujące uproszczenie - wiemy, że pełen cykl od początku do końca korytarza zajmuje nam `2*dlugosc + 2` kroków (korytarz-obrót-korytarz-obrót). Zatem jeśli mamy 1000 kroków i korytarz długości 4 (co daje nam cykl długości 10), wiemy od razu że wylądujemy na początku korytarza. Uproszczenie można zapisać w następujący sposób:

In [22]:
def robot(bateria, dlugosc):
    # Pozycja początkowa
    pozycja = 0
    # Kierunek (1=prawo, 0=lewo)
    kierunek = 1
    
    # Obliczamy maksymalne naladowanie - ze wzoru na sumę ciągu arytmetycznego byłoby to bateria*(bateria+1)//2
    naladowanie = sum([i for i in range(1, bateria+1)])
    
    # Zmniejszamy naladowanie o wszystkie cykle jakie wykonamy
    naladowanie = naladowanie % (2*dlugosc + 2)
    
    # Powtarzamy tak długo dopóki nie zużyjemy wszystkich kroków.
    while naladowanie > 0:
        # Idziemy w prawo
        if kierunek == 1:
            # Jeśli docieramy do skraju korytarza, zawracamy
            if pozycja == dlugosc:
                kierunek = 0
            # Jeśli nie, idziemy do przodu
            else:
                pozycja += 1
        # Idziemy w lewo
        else:
            # Jeśli dotarliśmy do skraju korytarza, zawracamy
            if pozycja == 0:
                kierunek = 1
            # Jeśli nie, idziemy do tyłu
            else:
                pozycja -= 1
        # Bateria rozładowuje nam się o krok
        naladowanie -= 1
    # Zwracamy pozycję.
    return pozycja

assert robot(5,10) == 6 
assert robot(6,7) == 5
assert robot(1,10) == 1
assert robot(1,100) == 1
assert robot(1,1000) == 1
assert robot(2,10) == 3
assert robot(2,100) == 3
assert robot(2,1000) == 3
assert robot(2,3) == 3
assert robot(2,2) == 2
assert robot(2,1) == 0
assert robot(3,6) == 6
assert robot(3,5) == 5
assert robot(3,4) == 3
assert robot(4,2) == 1

W tym momencie pojawia się kolejne uproszczenie - jeśli nasze naładowanie po operacji skracania jest mniejsze niż szerokość korytarza, to z automatu możemy zwrócić wartość naładowania. Jeśli natomiast byłoby ono większe niż szerokość korytarza, to wynikiem byłaby `dlugosc - (naladowanie - (dlugosc + 1)) = 2*dlugosc + 1 - naladowanie`. W ten sposób kompletnie pozbywamy się pętli:

In [25]:
def robot(bateria, dlugosc):
    # Obliczamy maksymalne naladowanie - ze wzoru na sumę ciągu arytmetycznego byłoby to bateria*(bateria+1)//2
    naladowanie = bateria*(bateria+1)//2
    # Zmniejszamy naladowanie o wszystkie cykle jakie wykonamy
    naladowanie = naladowanie % (2*dlugosc + 2)
    # Jeśli mamy ładunku mniej niż na cały korytarz, to zwracamy liczbę kroków.
    if naladowanie <= dlugosc:
        return naladowanie
    # Jeśli dotarliśmy do końca korytarza, to uwzględniamy fakt że musimy zawrócić i pozycja nam się nie zmienia
    elif naladowanie == dlugosc + 1:
        return naladowanie-1
    # Jeśli mamy więcej energii niż długość korytarza i zawracanie, zwracamy liczbę kroków z powrotem.
    else:
        return 2*dlugosc+1 - naladowanie
    
assert robot(5,10) == 6 
assert robot(6,7) == 5
assert robot(1,10) == 1
assert robot(1,100) == 1
assert robot(1,1000) == 1
assert robot(2,10) == 3
assert robot(2,100) == 3
assert robot(2,1000) == 3
assert robot(2,3) == 3
assert robot(2,2) == 2
assert robot(2,1) == 0
assert robot(3,6) == 6
assert robot(3,5) == 5
assert robot(3,4) == 3
assert robot(4,2) == 1

Na tym etapie możemy już uprawiać tylko code-golfing - zabawę w możliwie jak największe skracanie kodu.

## Projekt: rozwiązywanie Sudoku

Gra w Sudoku polega na uzupełnianiu częściowo zapełnionej planszy liczbami, tak by we wszystkich rzędach, kolumnach oraz kwadratach były wszystkie liczby od 1 do 9.

<img src="./sudoku.png" width=300/>

Błąd początkującego często polega na losowym uzupełnianiu planszy zgodnie z zasadami - niestety, tego typu podejście prowadzi bardzo szybko do rozwiązania sprzecznego. W większości pól z jest wiele możliwych rozwiązań, z których tylko jedno jest prawidłowe:

<img src="./sudoku_answers.png" width=300/>

W najprostszych wariantach zadań z Sudoku zazwyczaj można znaleźć pola, w których jest tylko jedno właściwe rozwiązanie. W bardziej skomplikowanych przypadkach rozwiązywanie szybko prowadzi do sytuacji, w której każde z pól ma wiele możliwych odpowiedzi, z których właściwą możemy wyłonić dopiero w drodze eliminacji.

### Zadania

1. Napisz funkcję która wczyta układ Sudoku z ciągu 81 cyfr (gdzie 0 będzie symbolizowało puste pole) i przetworzy na układ listy 9 list o długości 9 cyfr.
2. Napisz procedurę która w wygodny sposób wyświetli nam na ekranie bieżący układ Sudoku.
3. Napisz funkcje `kolumna, wiersz, kwadrat` które pozwolą nam się odwołać do danej kolumny, wiersza i kwadratu o podanym indeksie.
4. Napisz zestaw funkcji sprawdzający poprawność danej kolumny, danego wiersza, danego kwadratu. Poprawność jest określana jako brak powtórzeń żadnej cyfry.
5. Napisz funkcję sprawdzającą poprawność bieżącego układu Sudoku. Układ jest poprawny, kiedy każda kolumna, wiersz i kwadrat są poprawne.
6. Napisz funkcje które zwrócą listę liczb możliwych do wpisania do wiersza, kolumny lub kwadratu.
7. Napisz funkcję która zwróci listę liczb możliwą do wpisania w dane pole. Uwaga - liczby możliwe do wpisania muszą występować w liczbach możliwych dla wiersza, kolumny oraz kwadratu.
8. Napisz funkcję która zwróci wielkość listy liczb które można wpisać w dane pole.
9. Zmodyfikuj funkcję poprawności układu jako funkcję sprawdzającą poprawność wszystkich kwadratów, wierszy i kolumn oraz zapewniającą że każde puste pole ma przynajmniej jedną cyfrę możliwą do wpisania.
10. Napisz funkcję która automatycznie przetworzy dany układ Sudoku w układ uzupełniony o liczbę w polu z tylko jedną możliwością.